### Import modules

In [27]:
import tensorflow as tf
from keras.optimizers import SGD
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
tf.config.list_physical_devices(
device_type=None
)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
#!unzip /content/drive/MyDrive/archive.zip

### Hyper Parameters 

In [9]:
#Define some parameters
img_size = 150
batch_size = 16

### Image Data Generator
We will define image data generator

In [31]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.4, )
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.4)

## Build batch Generator

In [32]:
train_generator = train_datagen.flow_from_directory(
        'flowers',  # this is the target directory
        target_size=(img_size, img_size),  # all images will be resized to img_sizeximg_size
        batch_size=batch_size,
          color_mode="grayscale",
        class_mode='categorical',subset='training')

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'flowers',
        target_size=(img_size, img_size),
        batch_size=batch_size,
          color_mode="grayscale",
        class_mode='categorical', subset='validation')

Found 5191 images belonging to 6 classes.
Found 3455 images belonging to 6 classes.


In [33]:
class_names = train_generator.class_indices

### Build a CNN Model

In [34]:
model = keras.Sequential([
  keras.Input(shape=(img_size, img_size)),
  layers.Flatten(),
  layers.Dense(1024, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(1024, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(784, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(len(class_names), activation='softmax')
])

### Loss Function and Optimizer

In [35]:
opt = SGD(lr=0.01, momentum=0.9)

In [36]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 22500)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              23041024  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 784)               803600    
_________________________________________________________________
dropout_6 (Dropout)          (None, 784)              

### Model Training

In [38]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

In [39]:
model.fit_generator(train_generator,epochs=10,steps_per_epoch= STEP_SIZE_TRAIN, validation_data=validation_generator, validation_steps = STEP_SIZE_VALID)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


324/324 [==============================] - 37s 113ms/step - loss: 1.6462 - accuracy: 0.4621 - val_loss: 1.5011 - val_accuracy: 0.5006
Epoch 2/10
324/324 [==============================] - 36s 111ms/step - loss: 1.4774 - accuracy: 0.4953 - val_loss: 1.5207 - val_accuracy: 0.4788
Epoch 3/10
324/324 [==============================] - 36s 111ms/step - loss: 1.4646 - accuracy: 0.4966 - val_loss: 1.4948 - val_accuracy: 0.4988
Epoch 4/10
324/324 [==============================] - 36s 111ms/step - loss: 1.4747 - accuracy: 0.4865 - val_loss: 1.5218 - val_accuracy: 0.4797
Epoch 5/10
324/324 [==============================] - 36s 110ms/step - loss: 1.4557 - accuracy: 0.5002 - val_loss: 1.4834 - val_accuracy: 0.4959
Epoch 6/10
324/324 [==============================] - 36s 110ms/step - loss: 1.4577 - accuracy: 0.4971 - val_loss: 1.4933 - val_accuracy: 0.4878
Epoch 7/10
324/324 [==============================] - 36s 111ms/step - loss: 1.4728 - accuracy: 0.4834 - val_loss: 1.5277 - val_accuracy: 0.4

**Evaluation Step**


In [40]:
scores = model.evaluate_generator(validation_generator)
print("Accuracy = ", scores[1])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Accuracy =  0.49493488669395447
